# LangSmith 데이터셋 생성

자체 RAG평가용 데이터셋

1. Question -> Retriever

2. Question -> Anwer

3. Retriever -> Answer

`Question, Retrieval, Answer` 의 3가지 정보가 필요한 것이 일반적이지만, `Retrieval` 에 대한 `Ground Truth` 구축이 사실상 어렵습니다.

만약, `Retrieval` 에 대한 `Ground Truth` 가 존재한다면, 모두 데이터셋으로 저장하여 활용하고, 그렇지 않다면 `Question, Answer` 만으로 데이터셋을 구축하여 활용할 수 있습니다.


**정답과 llm의 답변을 비교하기 위한 output(answers)를 만드는것**

In [1]:
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


# 데이터셋 생성
`inputs`와 `outputs`를 활용하여 데이터셋을 생성. 데이터셋은 `question`과 `answer`로 구성

In [3]:
import pandas as pd

# 문서에대한 질문과 답변 목록
inputs = [
    "삼성전자가 만든 생성형 AI의 이름은 무엇인가요?",
    "미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행정명령을 발표한 날은 언제인가요?",
    "코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.",
]

# 문서에대한 질문에 대한 답변 목록
outputs = [
    "삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.",
    "2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.",
    "코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스 상태를 추적하고 투명성을 확보하기 위한 플랫폼입니다. 12개 기관과 협력하여 2,000여 개 데이터셋의 출처 정보를 제공하며, 개발자들이 데이터의 구성과 계보를 쉽게 파악할 수 있게 돕습니다.",
]

# 질문과 답변 쌍 생성
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]

# 데이터프레임으로 변환
df = pd.DataFrame(qa_pairs)

# 데이터프레임 출력
df.head()

,question,answer
0,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.
1,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.
2,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...


# HUGGING FACE에서 다운받아서 사용하는 방법

In [4]:
import pandas as pd
from datasets import load_dataset, Dataset
import os

# huggingface dataset에서 repo_id로 데이터셋 다운로드
dataset = load_dataset(
    "Jaehoya/ragas-test-dataset",
    token=os.environ["HUGGINGFACEHUB_API_TOKEN"], #private 데이터인 경우 필요
)

#데이터 셋에서 split한 기준으로 조회
huggingface_df = dataset["korean_v1"].to_pandas()
huggingface_df.head()

,contexts,evolution_type,metadata,episode_done,question_translated,ground_truth
0,"['£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\n...",simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,AI 개발의 맥락에서 콘텐츠 조정은 어떤 역할을 하나요?,콘텐츠 조정은 AI 시스템이 적절한 데이터로 학습하고 유해하거나 부적절한 콘텐츠가 ...
1,"['SPRi AI Brief |\n2023-12월호\nG7, 히로시마 AI 프로세스...",simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,G7이 수립한 AI 개발 가이드라인의 주요 내용은 무엇인가요?,G7이 마련한 AI 개발 가이드라인의 주요 내용은 첨단 AI 시스템에 대한 국제 행...
2,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n영국 AI 안...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,2023년 11월에 개최된 AI 안전 서밋의 목적은 무엇인가요?,2023년 11월에 개최된 AI 안전 서밋의 목적은 다양한 분야에서의 AI의 역할에...
3,"['구글 딥마인드, 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표\nKEY ...",simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,AGI 개발의 맥락에서 '운영화 진행'이란 무엇을 의미하나요?,AGI 개발의 맥락에서 '운영화 진행'이라는 용어는 AGI 기능의 발전을 구현하고 ...
4,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n알리바바 클라...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,통이췐웬 2.0은 이전 버전에 비해 어떤 점이 개선되었나요?,"통이췐웬 2.0은 이전 버전에 비해 MMLU, GSM8k, ARC-C 등 다양한 벤..."


# Langsmith 테스트를 위한 Dataset 생성

In [5]:
from langsmith import Client

client = Client()
dataset_name = "RAG_EVAL_DATASET_teddynote"

#데이터셋 생성 함수
def create_dataset(client, dataset_name, description=None):
    for dataset in client.list_datasets():
        if dataset.name == dataset_name:
            return dataset
        
    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description=description
    )
    return dataset

# 데이터셋 생성
dataset = create_dataset(client, dataset_name)

# 생성된 데이터셋에 예제 추가
client.create_examples(
    inputs=[{"question": q} for q in huggingface_df["question_translated"].tolist()],
    outputs=[{"answer": a} for a in huggingface_df["ground_truth"].tolist()],
    dataset_id = dataset.id,
)

데이터셋에 예제를 나중에 추가할 수 있습니다.

In [6]:
# 새로운 질문 목록
new_questions = [
    "삼성전자가 만든 생성형 AI의 이름은 무엇인가요?",
    "구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?",
]

# 새로운 답변 목록
new_answers = [
    "삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.",
    "사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 달러를 우선 투자하고 향후 15억 달러를 추가로 투자하기로 했습니다.",
]

# UI에서 업데이트된 버전 확인
client.create_examples(
    inputs=[{"question_translated": q} for q in new_questions],
    outputs=[{"ground_truth": a} for a in new_answers],
    dataset_id=dataset.id,
)